# Sentence to SQL demo

This notebook demos the sentence-to-SQL pipeline. Given a sentence naming some famous people, it generates and runs queries about those people against an appropriate database (Ian Limarta's Wikidata extracts).

## Supporting code

In [1]:
from functools import lru_cache
import gc
import json
import logging
from pathlib import Path
import re
import sqlite3
import string
from typing import Any, TypeVar, Union

from IPython.display import display, Markdown
import lark
import nest_asyncio
import requests
import spacy
import torch
import transformers
import vllm

from genparse import InferenceSetup, InferenceSetupVLLM

nest_asyncio.apply()

logger = logging.getLogger("notebook")

In [2]:
# Supporting code for entity extraction.
_SPACY_MODEL_NAME = "en_core_web_trf"
spacy_model = spacy.load(_SPACY_MODEL_NAME)
_PERSON_LABEL = "PERSON"

T = TypeVar('T')

def _uniquify(items: list[T]) -> list[T]:
    """
    O(n^2) order-preserving uniquification.

    Fine for short lists like a single sentence's list of PERSON entities.
    """
    result = []
    for item in items:
        if item not in result:
            result.append(item)
    return result

def get_people(sentence: str) -> list[str]:
    return _uniquify(
        [ent.text for ent in spacy_model(sentence).ents if ent.label_ == _PERSON_LABEL]
    )

In [3]:
people_query_template = string.Template(
    '''Write a SQL query such that the results will include any person whose name might be mentioned in the following sentence:

> $sentence

The database has one table, People, with columns as follows:

- given_name (the person's given name)
- family_name (the person's family name)

Prefer selecting all columns (with `*`) over listing every column individually.

The people named in the sentence are:

$people

Make your query as inclusive (permissive) as possible based on the names. Write a query that allows either a matching given name OR a matching family name. Where the full name is not given, use OR conditions, and consider that a name may be either a given or a family name. Write your query alone without additional commentary.'''
)

def _format_as_list(items: list[str]) -> str:
    """Format strings into a Markdown list (with no trailing newline)."""
    with_bullets = [f'- {item}' for item in items]
    return '\n'.join(with_bullets)

def format_people_prompt(sentence: str, people: list[str]) -> str:
    return people_query_template.substitute(sentence=sentence, people=_format_as_list(people))

In [4]:
_SQL_GRAMMAR_PATH = Path().resolve() / 'ian_wikidata_sql.lark'
assert _SQL_GRAMMAR_PATH.exists()
_SQL_GRAMMAR = _SQL_GRAMMAR_PATH.read_text(encoding='utf-8')
_BATCH_SIZE = 1
# Ben LeBrun's WIP server running on GCP as of 2024-07-11
_DEFAULT_GENPARSE_INFERENCE_SERVER_URI = 'http://34.122.30.137:8888/infer'


def get_best_posterior(posteriors: dict[str, float]) -> str:
    sorted_ = sorted(posteriors.items(), key=lambda x: (-x[1], x[0]))
    return sorted_[0]

def setup_inference(
    *,
    model_name: str,
    proposal: str = 'character',
    batch_size: int = _BATCH_SIZE,
) -> InferenceSetupVLLM:
    return InferenceSetupVLLM(model_name, grammar=_SQL_GRAMMAR, proposal=proposal, batch_size=batch_size)

def run_inference(
    prompt: str,
    *,
    inference_setup: InferenceSetupVLLM,
    max_new_tokens: int,
    n_particles: int,
) -> dict[str, float]:
    """
    Run inference locally.
    """
    return inference_setup(
        prompt=prompt,
        method='smc-standard',
        max_tokens=max_new_tokens,
        n_particles=n_particles,
        verbosity=0,
    ).posterior

def run_inference_server(
    prompt: str,
    *,
    model_name: str,
    proposal: str = 'character',
    batch_size: int = _BATCH_SIZE,
    max_tokens: int,
    n_particles: int,
    temperature: float = 1.0,
    grammar: str = _SQL_GRAMMAR,
    genparse_url: str = _DEFAULT_GENPARSE_INFERENCE_SERVER_URI,
) -> dict[str, float]:
    """
    Run inference using a server.
    """
    params = {
        'prompt': prompt,
        'method': 'smc-standard',
        'n_particles': n_particles,
        'lark_grammar': grammar,
        'proposal_name': proposal,
        'proposal_args': {},
        'max_tokens': max_tokens,
        'temperature': temperature,
    }
    headers = {
        "Content-type": "application/json",
        "Accept": "application/json"
    }
    response = requests.post(genparse_url, json=params, headers=headers)

    return response

In [5]:
_DB_PATH = Path("/home/joseph/chix/genfactdemo/data/people_large_ian_l_2024-06-27.sqlite")
assert _DB_PATH.is_file()

def run_people_query(query: str, *, db_path: Path = _DB_PATH) -> list[Any]:
    connection = sqlite3.connect(db_path)
    cursor = connection.cursor()

    clean_query = query.removesuffix('\u25aa').removesuffix('</s>')
    query_result = cursor.execute(clean_query).fetchall()
    column_names = [c[0] for c in cursor.description]
    if query_result:
        query_result = query_result[1:]
    connection.close()
    return [column_names, *query_result]

In [6]:
def _has_codefence_lines(text):
    return any(line.strip() == '```' for line in text.splitlines())

In [7]:
def _make_markdown_row(row):
    result = ''
    if row:
        parts = ['| ']
        for item in row:
            parts.append(str(item).strip())
            parts.append(' | ')
        result = ''.join(parts).rstrip()
    return result

def _make_separator_row(row):
    return _make_markdown_row(['-' * len(str(item).strip()) for item in row])

def display_query_results(query_result: list[Any]) -> None:
    if query_result:
        header, *rows = query_result
        parts = [_make_markdown_row(header), _make_separator_row(header)]
        parts.extend([_make_markdown_row(row) for row in rows])
        table_text = '\n'.join(parts + [''])
        display(Markdown(table_text))
    else:
        display(Markdown('(no results)'))

In [8]:
#GENPARSE_MODEL_NAME = 'codellama'
#inference_setup = setup_inference(model_name=GENPARSE_MODEL_NAME)
#inference_params = {'inference_setup': inference_setup, 'max_new_tokens': MAX_NEW_TOKENS, 'n_particles': N_PARTICLES}
#inference, likelihood = run_inference(prompt, **inference_params)

## Debugging

In [9]:
tokenizer = transformers.AutoTokenizer.from_pretrained('codellama/CodeLlama-7b-instruct-hf')

### Example inputs and outputs

| Sentence | Expected Output |
|:-------- |:--------------- |
| Dirac was offered the chance to study mathematics at Bristol without paying fees and he did so being awarded first class honours in 1923. | `SELECT ... FROM People WHERE "given_nameLabel.value" = 'Dirac' OR "family_nameLabel.value" = 'Dirac' [OR ...] [GROUP BY ...] [ORDER BY ...]`^1,8 |
| Mr Lee, a committed socialist from his student days, returned to Singapore where he became a prominent trade union lawyer. | `SELECT ... FROM People WHERE "given_nameLabel.value" = 'Lee' OR "family_nameLabel.value" = 'Lee' [OR ...] [GROUP BY ...] [ORDER BY ...]`^1 |
| After the failure of the rebellion, Sun Yat-sen, like many other anti-Qing revolutionaries, was forced into exile. | `SELECT ... FROM People WHERE "given_nameLabel.value" = 'Sun' AND "family_nameLabel.value" = 'Yat-sen' OR "family_nameLabel.value" = 'Sun' AND "given_nameLabel.value" = 'Yat-sen' [OR ...] [GROUP BY ...] [ORDER BY ...]`^2 |
| Gandhi grew up in a home steeped in religion, and he took for granted religious tolerance and the doctrine of ahimsa (noninjury to all living beings). | `SELECT ... FROM People WHERE "given_nameLabel.value" = 'Gandhi' OR "family_nameLabel.value" = 'Gandhi' [OR ...] [GROUP BY ...] [ORDER BY ...]` |
| Chamberlain reached out to the opposition in the hope of forming a coalition, which Labour leader Clement Attlee was prepared to do, but not with Chamberlain as Prime Minister. | `SELECT ... FROM People WHERE "given_nameLabel.value" = 'Clement' AND "family_nameLabel.value" = 'Attlee' OR "given_nameLabel.value" = 'Chamberlain' OR "family_nameLabel.value = 'Chamberlain' [OR ...] [GROUP BY ...] [ORDER BY ...]` |
| Eleanor became the first first lady to travel to an active war zone when she undertook a month-long journey to the warfront in the Pacific. | `SELECT ... FROM People WHERE "given_nameLabel.value" = 'Eleanor' OR "family_nameLabel.value" = 'Eleanor' [OR ...] [GROUP BY ...] [ORDER BY ...]`^7 |
| In 1991, Martha Stewart, Inc., became Martha Stewart Living Omnimedia, Inc., with the release of her magazine, Martha Stewart Living. | `SELECT ... FROM People WHERE "given_nameLabel.value" = 'Martha' AND "family_nameLabel.value" = 'Stewart' [OR ...] [GROUP BY ...] [ORDER BY ...]` |
| By the late 1990s, however, Kariko's work on mRNA and gene therapy had stalled—again, for lack of funding. | `SELECT ... FROM People WHERE "given_nameLabel.value" = 'Kariko' OR "family_nameLabel.value" = 'Kariko' [OR ...] [GROUP BY ...] [ORDER BY ...]`^5 |
| Clarkson did not use the same team of songwriters as she had in the past and relied more strongly on songs that she co-wrote. | `SELECT ... FROM People WHERE "given_nameLabel.value" = 'Clarkson' OR "family_nameLabel.value" = 'Clarkson' [OR ...] [GROUP BY ...] [ORDER BY ...]`^6
| Lovelace was deeply intrigued by Babbage’s plans for a tremendously complicated device he called the Analytical Engine, which was to combine the array of adding gears of his earlier Difference Engine with an elaborate punchcard operating system. | `SELECT ... FROM People WHERE "given_nameLabel.value" = 'Lovelace' OR "family_nameLabel.value" = 'Lovelace' OR "given_nameLabel.value" = 'Babbage' OR "family_nameLabel.value" = 'Babbage' [OR ...] [GROUP BY ...] [ORDER BY ...]`^7 |

Some judgment calls:

1. We want to get all the possible matches, rather than the likely intended referent of a name. That is: Refrain from inferring (with certainty) that Dirac means *Paul* Dirac the mathematician, or that Mr. Lee necessarily means *Lee Kuan* Yew the first Prime Minister of Singapore. Likewise, refrain from inferring we necessarily want *Mohandas* (or titular Mahatma) Gandhi, *Neville* Chamberlain, Eleanor *Roosevelt*, *Katalin* Kariko (if its training data is recent enough), *Kelly* Clarkson, *Augusta Ada King, Countess of* Lovelace, and *Charles* Babbage. If the full name isn't specified, we won't try to guess it.
2. Returning extra results (e.g. querying for Truman and Churchill in addition to Eleanor Roosevelt) is fine as long as we the query ORs them with the important conditions, rather than ANDing them.
3. Strengthening the conditions with AND terms is bad.
4. Ordering and grouping don't matter.
5. The names are stored in the database as ASCII. So, Kariko rather than Karikó.
6. We don't attempt to filter on anything except the names. In theory, we could try to filter on the occupation here, but it might be coded as 'writer' or more narrowly as 'songwriter' or as 'singer'... or any number of other things. I'm sure Wikidata has lots of possible occupation Qnodes and any given entry might or might not link against all the relevant ones, so this just seems like a chance to shoot ourselves in the foot and exclude valid matches.
7. When a sentence names multiple people, we query for all named people at once, rather than writing separate queries for each person.
8. We DO want to allow the inference that Bristol is not the name of a person.
9. Names have a bunch of weird edge cases that we are going to ignore for now. *Ada Lovelace* is just one example: Her first name wasn't Ada (it was Augusta) and her family name wasn't Lovelace (it was King). A related challenge is the ordering of given name vs. family name -- is it the conventional Western ordering (given THEN family) or something else?

### Thoughts

#### We should provide only the relevant name columns in the prompt

In theory, it would be nice to include all the columns in the LM's prompt so that it can use whatever is relevant to filter the results. In practice, Code Llama filters in stupid ways that exclude valid results.

Here's an example. Given this sentence:

```
Dirac was offered the chance to study mathematics at Bristol without paying fees and he did so being awarded first class honours in 1923.
```

Even when instructed to write an inclusive, permissive query that hard-filters only on the name, Code Llama frequently generates queries that specify conditions like `WHERE ("given_nameLabel.value" = 'Dirac' OR "family_nameLabel.value" = 'Dirac') AND ("occupationLabel.value" = 'physicist')`. Sometimes it generates even dumber conditions, like `"date_of_birthLabel.value" = '1923'` --- obviously he wasn't born in 1923, babies don't get awarded first-class honors in university (implied to be graduate-level) mathematics, it's just a year that happens to be zmentioned in the sentence.

To avoid this, I plan to name ONLY the given and family name columns in the prompt. If we want to refine the query to be more specific, we can do that in later steps. For the first query generation, I want to generate a highly permissive query.

In [ ]:
# The old template, which includes every column name
string.Template('''Write a SQL query such that the results will include any person whose name might be mentioned in the following sentence:

> $sentence

Make your query as inclusive (permissive) as possible based on the names. Your query must allow ANY row that has a matching name regardless of the other columns. If the full name is not given, write a query that allows either a matching given name OR a matching family name. Use AND only to specify the full name if it is explicitly given in the sentence. Where the full name is not given, use OR conditions conditions, and consider that a name may be either a given or a family name. Write your query alone without additional commentary.

The database has one table, People, with columns as follows:

- "given_nameLabel.value" (the person's given name)
- "family_nameLabel.value" (the person's family name)
- "name_in_native_languageLabel.value" (name as written in native language)
- "languages_spoken__written_or_signedLabel.value" (a language they speak, write, or sign -- always capitalized)
- "date_of_birthLabel.value" (their date of birth, in YYYY-MM-DDTHH:mm:ss+XX:ZZ form)
- "place_of_birthLabel.value" (their place of birth)
- "spouseLabel.value" (their spouse's common name)
- "motherLabel.value" (their mother's common name)
- "fatherLabel.value" (their father's common name)
- "country_of_citizenshipLabel.value" (country they are a citizen of)
- "occupationLabel.value" (their occupation)
- "religion_or_worldviewLabel.value" (their religion or worldview)
- "sex_or_genderLabel.value" (their sex or gender)

The dots are part of the column names and so the column names must be quoted. Prefer selecting all columns (with `*`) over listing every column individually.

The people named in the sentence are:

$people''')

#### Code Llama's preferred SQL output structure

Code Llama seems to prefer to:

1. prefix its output with spaces
2. separate each part of the query with a newline (e.g. `  SELECT *\nFROM People\nWHERE condition1\nOR condition2...`, or `  SELECT *\nFROM People\nWHERE     condition1\n    OR\n    condition2...`)
3. end its query with a semicolon
4. sometimes use `LIKE` instead of `=`
5. not wrap its `WHERE` conditions in parentheses (e.g it prefers `WHERE condition1 OR condition2 OR condition3` to `WHERE (condition1 OR (condition2 OR condition3))`).
6. group arbitrary subsets of its `WHERE` conditions in parentheses

### Running the language model unconstrained

In [10]:
try:
    del model
    gc.collect()
    torch.cuda.empty_cache()
except NameError:
    pass

In [11]:
# Taken from a magic number in the Genparse VLLM backend code.
_MAX_MODEL_LEN = 4096
# Why dtype=torch.float32? It's in the Genparse VLLM backend code.
model = vllm.LLM('codellama/CodeLlama-7b-instruct-hf', dtype=torch.float32, max_model_len=_MAX_MODEL_LEN)

INFO 07-15 18:32:13 config.py:1214] Upcasting torch.bfloat16 to torch.float32.
INFO 07-15 18:32:13 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='codellama/CodeLlama-7b-instruct-hf', speculative_config=None, tokenizer='codellama/CodeLlama-7b-instruct-hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float32, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=codellama/CodeLlama-7b-instruct-hf)
INFO 07-15 18:32:14 selector.py:136] Cannot use FlashAttention-2 backend for dtype other than torch.float16 or torch.bfloat16.
INFO 07-15 18:32:14 selector.py:51] Using XFormer

In [12]:
sentence = "After the failure of the rebellion, Sun Yat-sen, like many other anti-Qing revolutionaries, was forced into exile."
people = get_people(sentence)
prompt = tokenizer.apply_chat_template([{'role': 'user', 'content': format_people_prompt(sentence=sentence, people=people)}], tokenize=False)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')

sampling_params = vllm.SamplingParams(temperature=1.0, max_tokens=1024, n=25)
response = model.generate(prompt, sampling_params=sampling_params)[0]
for i, output in enumerate(response.outputs, start=1):
    print(f'Response {i} token IDs: {output.token_ids}')
    decodes = [tokenizer.decode(id_) for id_ in output.token_ids]
    print(f'Response {i} token decodes: {decodes}')
    print(f'Generated Query {i}: ```{output.text}```')

Prompt (215 tokens): ```<s>[INST] Write a SQL query such that the results will include any person whose name might be mentioned in the following sentence:

> After the failure of the rebellion, Sun Yat-sen, like many other anti-Qing revolutionaries, was forced into exile.

The database has one table, People, with columns as follows:

- given_name (the person's given name)
- family_name (the person's family name)

Prefer selecting all columns (with `*`) over listing every column individually.

The people named in the sentence are:

- Sun Yat-sen

Make your query as inclusive (permissive) as possible based on the names. Write a query that allows either a matching given name OR a matching family name. Where the full name is not given, use OR conditions, and consider that a name may be either a given or a family name. Write your query alone without additional commentary. [/INST]```


Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.77s/it, est. speed input: 19.96 toks/s, output: 193.80 toks/s]

Response 1 token IDs: [29871, 5097, 334, 3895, 11647, 5754, 2183, 29918, 978, 353, 525, 29903, 348, 612, 271, 29899, 4881, 29915, 6323, 3942, 29918, 978, 353, 525, 29903, 348, 612, 271, 29899, 4881, 2670, 2]
Response 1 token decodes: ['', 'SELECT', '*', 'FROM', 'People', 'WHERE', 'given', '_', 'name', '=', "'", 'S', 'un', 'Y', 'at', '-', 'sen', "'", 'OR', 'family', '_', 'name', '=', "'", 'S', 'un', 'Y', 'at', '-', 'sen', "';", '</s>']
Generated Query 1: ```  SELECT * FROM People WHERE given_name = 'Sun Yat-sen' OR family_name = 'Sun Yat-sen';```
Response 2 token IDs: [29871, 5097, 334, 13, 21482, 11647, 13, 22043, 2183, 29918, 978, 353, 525, 29903, 348, 612, 271, 29899, 4881, 29915, 13, 1955, 3942, 29918, 978, 353, 525, 29903, 348, 612, 271, 29899, 4881, 29915, 2]
Response 2 token decodes: ['', 'SELECT', '*', '\n', 'FROM', 'People', '\n', 'WHERE', 'given', '_', 'name', '=', "'", 'S', 'un', 'Y', 'at', '-', 'sen', "'", '\n', 'OR', 'family', '_', 'name', '=', "'", 'S', 'un', 'Y', 'at', '-

### Growing the grammar slowly

In [13]:
smallgrammar = r'''
// start: FREE_TEXT CODEFENCE sql_query CODEFENCE FREE_TEXT [EOS]
//start: (sql_query | rambling) [EOS | NL | "▪"]
//rambling: FREE_TEXT CODEFENCE sql_query CODEFENCE FREE_TEXT
//FREE_TEXT: /([^`]|`[^`]+`|``[^`]+``)*[^`]/
start: sql_query [EOS]
sql_query: "SELECT" SPACE+ STAR WS_SEP "FROM" SPACE+ "People" WS_SEP "WHERE" WS_SEP bool_condition_people [";"]
EOS: "</s>"
CODEFENCE: "```"

from_expr_people: "People"

//bool_condition_people: bool_expr_people | bool_condition_people WS_SEP "OR" WS_SEP bool_condition_people | bool_condition_people WS_SEP "AND" WS_SEP bool_condition_people | "(" (NL? SPACE*) bool_condition_people (NL? SPACE*) ")"
bool_condition_people: and_expr_people
and_expr_people: or_expr_people | "(" NL? SPACE* or_expr_people NL? SPACE* ")" WS_SEP ( "AND" WS_SEP "(" NL? SPACE* or_expr_people NL? SPACE* ")" )+
or_expr_people: bool_expr_people | "(" NL? SPACE* or_expr_people NL? SPACE* ")" | bool_expr_people WS_SEP "OR" WS_SEP or_expr_people
bool_expr_people: people_column SPACE* "=" SPACE* STRING | people_column SPACE+ "LIKE" SPACE+ STRING

people_column: _given_name | _family_name
_given_name: "given_name"
_family_name: "family_name"

STAR: "*"
people_value: STRING
STRING: "'" CHAR * "'"
LETTER: "a".."z" | "A".."Z"
BACKSLASH: "\\"
SYMBOL: "-" | "(" | ")" | "[" | "]" | "{" | "}" | "!" | "@" | "%" | "*" | "." | "," | ":" | ";" | "<" | "=" | ">" | "/" | "?" | "_" | "`"
DIGIT: "0".."9"
CHAR: "''" | LETTER | DIGIT | SYMBOL | SPACE | (BACKSLASH BACKSLASH)
SPACE: " "
WS_SEP: SPACE+ | INDENT
INDENT: NL SPACE*
WS: " " | NL
NL: "\n"
'''

sentence = "After the failure of the rebellion, Sun Yat-sen, like many other anti-Qing revolutionaries, was forced into exile."
people = get_people(sentence)
prompt = tokenizer.apply_chat_template(
    [
        {'role': 'user', 'content': format_people_prompt(sentence=sentence, people=people)}
    ],
    tokenize=False,
)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')


# ignored top_p=0.95
server_inference_params = {'model_name': 'codellama', 'max_tokens': 256, 'n_particles': 15, 'temperature': .8, 'grammar': smallgrammar}
response = run_inference_server(prompt, **server_inference_params)
try:
    data = response.json()
    for i, (query, likelihood) in enumerate(data['posterior'].items(), start=1):
        print(f'Generated Query {i} (likelihood {100 * likelihood:.2f}%): ```{query}```')
except json.JSONDecodeError:
    print(response.status_code)
    print(response.text)

Prompt (215 tokens): ```<s>[INST] Write a SQL query such that the results will include any person whose name might be mentioned in the following sentence:

> After the failure of the rebellion, Sun Yat-sen, like many other anti-Qing revolutionaries, was forced into exile.

The database has one table, People, with columns as follows:

- given_name (the person's given name)
- family_name (the person's family name)

Prefer selecting all columns (with `*`) over listing every column individually.

The people named in the sentence are:

- Sun Yat-sen

Make your query as inclusive (permissive) as possible based on the names. Write a query that allows either a matching given name OR a matching family name. Where the full name is not given, use OR conditions, and consider that a name may be either a given or a family name. Write your query alone without additional commentary. [/INST]```
Generated Query 1 (likelihood 9.89%): ```SELECT * FROM People
WHERE given_name = 'Sun'
OR family_name = 'Sun'

In [24]:
init = """SELECT *
FROM People
WHERE given_name = 'Sun Yat-sen' OR family_name = 'Sun Yat-sen'
OR family_name='Yat-sen' OR given_name='Yat-sen'
OR family_name = 'Yat-Sen' OR given_name = 'Yat-Sen'
OR given_name='Sun' OR family_name='Sun'"""
print(f'Tree: {lark.Lark(smallgrammar).parse(init).pretty()}')
# check p_next()
from genparse import BoolCFGLM
from genparse.util import LarkStuff

guide = BoolCFGLM(LarkStuff(smallgrammar).char_cfg())
guide.p_next(init + "\n")

Tree: start
  sql_query
     
    *
    

     
    

     
    bool_condition_people
      and_expr_people
        or_expr_people
          bool_expr_people
            people_column
             
             
            'Sun Yat-sen'
           
           
          or_expr_people
            bool_expr_people
              people_column
               
               
              'Sun Yat-sen'
            

             
            or_expr_people
              bool_expr_people
                people_column
                'Yat-sen'
               
               
              or_expr_people
                bool_expr_people
                  people_column
                  'Yat-sen'
                

                 
                or_expr_people
                  bool_expr_people
                    people_column
                     
                     
                    'Yat-Sen'
                   
                   
                  or_expr_people
                 

key,value
O,1
,1


In [ ]:
parser = lark.Lark(smallgrammar)
valid_queries = [
    '''  SELECT *
FROM People
WHERE given_name = 'Sun Yat-sen'
OR family_name = 'Sun Yat-sen';''',
    '''  SELECT * FROM People WHERE given_name = 'Sun Yat-sen' OR family_name = 'Sun Yat-sen';''',
    '''  SELECT *
FROM People
WHERE given_name = 'Sun Yat-sen' OR family_name = 'Sun Yat-sen\'''',
    '''  SELECT *
FROM People
WHERE (given_name LIKE '%Sun%' OR family_name LIKE '%Sun%')
AND (given_name LIKE '%Yat-sen%' OR family_name LIKE '%Yat-sen%');''',
    '''  SELECT *
FROM People
WHERE given_name LIKE '%Sun%' OR family_name LIKE '%Sun%';''',
    '''  SELECT *
FROM People
WHERE given_name = 'Sun'
OR family_name = 'Yat-sen'
OR given_name = 'Yat-sen'
OR family_name = 'Sun\'''',
    '''  SELECT * FROM People
WHERE given_name LIKE '%Sun%' OR family_name LIKE '%Sun%' OR given_name LIKE '%Yat%' OR family_name LIKE '%Yat%' OR given_name LIKE '%sen%' OR family_name LIKE '%sen%';''',
    '''  SELECT * FROM People WHERE (given_name = 'Sun' OR family_name = 'Sun') OR (given_name = 'Yat-sen' OR family_name = 'Yat-sen')''',
    '''  SELECT * FROM People WHERE given_name = 'Sun' OR given_name = 'Yat-sen' OR family_name = 'Sun' OR family_name = 'Yat-sen\'''',
    '''  SELECT * FROM People
WHERE
  given_name = 'Sun Yat-sen'
  OR family_name = 'Sun Yat-sen'
  OR given_name = 'Yat-sen'
  OR family_name = 'Yat-sen';''',
    '''  SELECT *
FROM People
WHERE given_name LIKE '%Sun%' OR family_name LIKE '%Sun%'
LIMIT 1;''',
    '''  SELECT *
FROM People
WHERE given_name LIKE '%yat-sen%' OR family_name LIKE '%yat-sen%';''',
    '''  SELECT *
FROM People
WHERE
    given_name = 'Sun' OR family_name = 'Yat-sen' OR
    given_name = 'Yat-sen' OR family_name = 'Sun';''',
    '''  SELECT * FROM People WHERE given_name = 'Sun Yat-sen'
OR family_name = 'Sun Yat-sen';''',
    '''  SELECT * FROM People
WHERE given_name = 'Sun Yat-sen'
OR family_name = 'Sun Yat-sen\'''',
    '''  SELECT * FROM People
WHERE given_name = 'Sun'
OR family_name = 'Yat-sen'
OR given_name LIKE '%-Yat-sen'
OR family_name LIKE '%-Yat-sen';''',
    '''  ```
SELECT *
FROM People
WHERE given_name LIKE '%Sun%'
OR family_name LIKE '%Sun%';
```''',
    '''  SELECT * FROM People WHERE
given_name = "Sun" OR
given_name = "Yat-sen" OR
family_name = "Sun" OR
family_name = "Yat-sen"''',
    '''  SELECT
*
FROM
People
WHERE
(given_name LIKE '%Sun%' OR family_name LIKE '%Sun%')
AND
(given_name LIKE '%Yat%' OR family_name LIKE '%Yat%')
AND
(given_name LIKE '%sen%' OR family_name LIKE '%sen%')''',
    '''  SELECT *
FROM People
WHERE
    (given_name = 'Sun Yat-sen'
    OR family_name = 'Sun Yat-sen')
    OR given_name LIKE '%Yat-sen%'
    OR family_name LIKE '%Yat-sen%';''',
    '''  SELECT * FROM People
WHERE
(given_name LIKE '%Yat-sen%' OR family_name LIKE '%Yat-sen%')
OR
(given_name LIKE '%Sun%' OR family_name LIKE '%Sun%');''',
    '''  SELECT * FROM People
WHERE (given_name LIKE '%Sun%' OR family_name LIKE '%Sun%') AND (given_name LIKE '%Yat-sen%' OR family_name LIKE '%Yat-sen%');

''',
    '''  ```
SELECT * FROM People
WHERE given_name = 'Sun Yat-sen'
OR family_name = 'Sun Yat-sen'
```
This query will select all rows in the `People` table where the value in the `given_name` column matches the name 'Sun Yat-sen' or the value in the `family_name` column matches the name 'Sun Yat-sen'. This will include any person whose name might be mentioned in the sentence provided.''',
    '''  SELECT *
    FROM People
    WHERE (
        given_name = 'Sun Yat-sen'
        OR family_name = 'Sun Yat-sen'
    )
    OR (
        given_name = 'many other anti-Qing revolutionaries'
        OR family_name = 'many other anti-Qing revolutionaries'
    )
    OR (
        given_name = 'rebellion'
        OR family_name = 'rebellion'
    )
    OR (
        given_name = 'after'
        OR family_name = 'after'
    )
    OR (
        given_name = 'exile'
        OR family_name = 'exile'
    )
    ;''',
    '''  Here is a SQL query that will retrieve any person who might be mentioned in the given sentence:
```
SELECT *
FROM People
WHERE given_name LIKE '%Sun%' OR family_name LIKE '%Sun%' OR given_name LIKE '%Yat-sen%';
```
This query retrieves all columns from the `People` table where either the `given_name` column or the `family_name` column contains the string 'Sun' or 'Yat-sen', or where the `given_name` column contains any string that ends with '-sen'.

This query is inclusive, as it allows for both given and family names to be used, and assumes that a name may be either a given or a family name, so it uses OR conditions to accommodate both possibilities.

Note that this query will retrieve any person whose name might be mentioned in the sentence, not just Sun Yat-sen.''',
]

n_valid = 0
n_probably_valid = 0
n_with_backtick = 0
n_with_or_sandwich = 0
n_with_limit = 0
n_with_dqs = 0
n_with_newline_after_keyword = 0
for query in valid_queries:
    print(f'Query: ```{query}``` ')
    if "`" in query:
        print("BACKTICK")
        n_with_backtick += 1
    if re.search(r"\)[ \n]+OR[ \n]+\(", query) or re.search(r"[ \n]+OR[ \n]+\(", query) or re.search(r"\)[ \n]+OR[ \n]+", query):
        print("OR-SANDWICH")
        n_with_or_sandwich += 1
    if re.search(r"\bLIMIT\b", query):
        print("LIMITED!")
        n_with_limit += 1
    if re.search(r" *= \"[^\"]+\"", query):
        print("DOUBLE QUOTES STRING!")
        n_with_dqs += 1
    if "SELECT\n" in query or "FROM\n" in query:
        print("NEWLINE AFTER SELECT OR FROM")
        n_with_newline_after_keyword += 1
    try:
        parser.parse(query)
        print("VALID\n")
        n_valid += 1
    except (lark.exceptions.ParseError, lark.exceptions.UnexpectedCharacters) as e:
        if 'Unexpected end-of-input.' in str(e) and '* EOS\n' in str(e) or (re.match(r"No terminal matches '[ \n]+'", str(e)) and query.rstrip() != query):
            print("PROBABLY VALID?\n")
            n_probably_valid += 1
        else:
            print("INVALID:")
            print(e)
            print()
print(f'{n_valid}/{len(valid_queries)} parsed correctly')
print(f'{n_valid + n_probably_valid}/{len(valid_queries)} likely allowed by grammar')
print(f'{n_with_backtick} have a backtick')
print(f'{n_with_or_sandwich} have OR sandwich')
print(f'{n_with_limit} have LIMIT')
print(f'{n_with_dqs} have "" string')
print(f'{n_with_newline_after_keyword} have a newline after SELECT or FROM')
# two other singleton errors:
#    1. newline immediately after SELECT or FROM (or WHERE?)
#    2. OR-sandwich except one side of the sandwich (in this case, the right side) is not parenthesized.

In [ ]:
import lark
testgrammar = r'''start: WS+ EOS
EOS: "</s>"
WS: " " | "\n"
%ignore WS
'''
parser = lark.Lark(testgrammar)
parser.parse(' \n   </s>')

## Demo

Example sentences:

1. `George Washington went to high school with Tim O'Donnell.`
2. `Eleanor became the first first lady to travel to an active war zone when she undertook a month-long journey to the warfront in the Pacific.`

In [ ]:
GENPARSE_MODEL_NAME = 'codellama'
MAX_NEW_TOKENS = 512
N_PARTICLES = 15
server_inference_params = {'model_name': GENPARSE_MODEL_NAME, 'max_tokens': MAX_NEW_TOKENS, 'n_particles': N_PARTICLES}

while True:
    sentence = input('Give me a sentence involving a (named) famous person: ')
    if sentence == 'q':
        break

    print(f'Processing sentence `{sentence}`...')
    people = get_people(sentence)
    logger.debug(f'People: {people}')
    
    prompt = format_people_prompt(sentence=sentence, people=people)
    logger.debug(f'Prompt: {prompt}')
    
    inference, likelihood = get_best_posterior(run_inference_server(prompt, **server_inference_params)['posterior'])
    display(Markdown('Query:'))
    assert not _has_codefence_lines(inference)
    display(Markdown(f'```\n{inference}\n```'))
    display(Markdown(f'Likelihood: {likelihood}'))
    
    try:
        results = run_people_query(inference)
        display(Markdown('Query Results:'))
        display_query_results(results)
    except sqlite3.ProgrammingError as e:
        display(Markdown(f'Error in query: {e}'))
    except sqlite3.OperationalError as e:
        display(Markdown(f'Incomplete query generated: {e}'))

Give me a sentence involving a (named) famous person:  George Washington went to high school with Tim O'Donnell.


Processing sentence `George Washington went to high school with Tim O'Donnell.`...
